<a href="https://colab.research.google.com/github/YesianaPhan/News-Classifier-Fastai/blob/main/News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#News Dataset Classifier using fastai library

In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
     |████████████████████████████████| 704.8MB 25kB/s 


In [2]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = '/content/gdrive/My Drive/'
path=Path(root_dir)

Mounted at /content/gdrive


In [4]:
df = pd.read_csv(path/"news.csv")
df = df.dropna()
df = df.reset_index(drop = True)
print(df.shape)
df.head()

(100, 3)


,title,text,category
0,moeldoko harap seni budaya bisa jadi magnet ku...,Kepala Staf Kepresidenan RI Jenderal TNI (Pur...,Culture
1,seniman palestina sulap tembakan israel jadi k...,"Berbeda dari seniman pada umumnya, seorang se...",Culture
2,sontek cara 56 kreator berpengaruh di dunia ci...,", menjadi modal utama yang harus dimiliki tia...",Culture
3,intip pertunjukan fashion dalam balutan seni t...,"Dari Sabang sampai Merauke, Indonesia menawar...",Culture
4,kenang benyamin sueb teater abnon gelar pement...,Benyamin Suaeb bukan hanya sekadar nama. Soso...,Culture


In [5]:
df["text_lower"] = df["text"].str.lower()
df["text_no_char"] = df["text_lower"].str.replace(r'[^a-zA-Z\s]+', '')
df.head()

,title,text,category,text_lower,text_no_char
0,moeldoko harap seni budaya bisa jadi magnet ku...,Kepala Staf Kepresidenan RI Jenderal TNI (Pur...,Culture,kepala staf kepresidenan ri jenderal tni (pur...,kepala staf kepresidenan ri jenderal tni purn...
1,seniman palestina sulap tembakan israel jadi k...,"Berbeda dari seniman pada umumnya, seorang se...",Culture,"berbeda dari seniman pada umumnya, seorang se...",berbeda dari seniman pada umumnya seorang sen...
2,sontek cara 56 kreator berpengaruh di dunia ci...,", menjadi modal utama yang harus dimiliki tia...",Culture,", menjadi modal utama yang harus dimiliki tia...",menjadi modal utama yang harus dimiliki tiap...
3,intip pertunjukan fashion dalam balutan seni t...,"Dari Sabang sampai Merauke, Indonesia menawar...",Culture,"dari sabang sampai merauke, indonesia menawar...",dari sabang sampai merauke indonesia menawark...
4,kenang benyamin sueb teater abnon gelar pement...,Benyamin Suaeb bukan hanya sekadar nama. Soso...,Culture,benyamin suaeb bukan hanya sekadar nama. soso...,benyamin suaeb bukan hanya sekadar nama sosok...


In [6]:
pip install Sastrawi

     |████████████████████████████████| 215kB 8.0MB/s 


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords1 = set(stopwords.words('indonesian'))
print(stopwords1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'beginian', 'sekalian', 'berturut', 'sesudah', 'sekadarnya', 'setengah', 'diperkirakan', 'bawah', 'terlalu', 'menginginkan', 'berlangsung', 'terakhir', 'untuk', 'segera', 'diakhirinya', 'mendapatkan', 'seketika', 'jelaskan', 'berkata', 'bung', 'wong', 'begini', 'diri', 'asal', 'belakangan', 'dikatakan', 'masing-masing', 'sedang', 'terhadapnya', 'enggaknya', 'menuturkan', 'terjadinya', 'menghendaki', 'sesama', 'atas', 'semata-mata', 'sepantasnya', 'semakin', 'terjadilah', 'amatlah', 'jika', 'tempat', 'ibu', 'ingin', 'pantas', 'percuma', 'berkeinginan', 'dini', 'sinilah', 'pertanyakan', 'berakhirlah', 'masihkah', 'guna', 'bermacam', 'bekerja', 'waktu', 'bersama', 'sejak', 'ujar', 'kamilah', 'masalah', 'cara', 'mengatakannya', 'ada', 'dimaksudkan', 'gunakan', 'mungkin', 'ditunjukkannya', 'usah', 'seolah', 'para', 'tambah', 'selalu', 'sedemikian', 'seperti', 'agar', 'dipunyai', '

In [8]:
def rmv_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords1])
df["text_no_stop"] = df["text_no_char"].apply(lambda text: rmv_stopwords(text))
df.head()

,title,text,category,text_lower,text_no_char,text_no_stop
0,moeldoko harap seni budaya bisa jadi magnet ku...,Kepala Staf Kepresidenan RI Jenderal TNI (Pur...,Culture,kepala staf kepresidenan ri jenderal tni (pur...,kepala staf kepresidenan ri jenderal tni purn...,kepala staf kepresidenan ri jenderal tni purn ...
1,seniman palestina sulap tembakan israel jadi k...,"Berbeda dari seniman pada umumnya, seorang se...",Culture,"berbeda dari seniman pada umumnya, seorang se...",berbeda dari seniman pada umumnya seorang sen...,berbeda seniman seniman majdy abu tagiyya menc...
2,sontek cara 56 kreator berpengaruh di dunia ci...,", menjadi modal utama yang harus dimiliki tia...",Culture,", menjadi modal utama yang harus dimiliki tia...",menjadi modal utama yang harus dimiliki tiap...,modal utama dimiliki orang berkembang bidang d...
3,intip pertunjukan fashion dalam balutan seni t...,"Dari Sabang sampai Merauke, Indonesia menawar...",Culture,"dari sabang sampai merauke, indonesia menawar...",dari sabang sampai merauke indonesia menawark...,sabang merauke indonesia menawarkan keindahan ...
4,kenang benyamin sueb teater abnon gelar pement...,Benyamin Suaeb bukan hanya sekadar nama. Soso...,Culture,benyamin suaeb bukan hanya sekadar nama. soso...,benyamin suaeb bukan hanya sekadar nama sosok...,benyamin suaeb nama sosok seniman asli betawi ...


In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer = StemmerFactory().create_stemmer()

def stemming(text):
  return " ".join([stemmer.stem(word) for word in text.split()])

df["clean_data"] = df["text_no_stop"].apply(lambda text: stemming(text))

In [10]:
df = df[["title","clean_data", "category"]]
df.columns = ["title","text","category"]
df.head()

,title,text,category
0,moeldoko harap seni budaya bisa jadi magnet ku...,kepala staf presiden ri jenderal tni purn dr m...,Culture
1,seniman palestina sulap tembakan israel jadi k...,beda seniman seniman majdy abu tagiyya cipta k...,Culture
2,sontek cara 56 kreator berpengaruh di dunia ci...,modal utama milik orang kembang bidang gelut l...,Culture
3,intip pertunjukan fashion dalam balutan seni t...,sabang merauke indonesia tawar indah alam puka...,Culture
4,kenang benyamin sueb teater abnon gelar pement...,benyamin suaeb nama sosok seniman asli betawi ...,Culture


In [11]:
input_train = df.loc[30:55, 'text']
target_train = df.loc[30:55, 'category']
input_val = df.loc[70:100, 'text']
target_val = df.loc[70:100, 'category']

In [12]:
train_texts = input_train.values
train_labels = target_train.values
val_texts = input_val.values
val_labels = target_val.values
np.random.seed(15)

train_idx = np.random.permutation(len(train_texts))
val_idx = np.random.permutation(len(val_texts))

train_texts = train_texts[train_idx]
val_texts = val_texts[val_idx]
train_labels = train_labels[train_idx]
val_labels = val_labels[val_idx]
col_names = ['text','category']

train = pd.DataFrame({'text':train_texts, 'category':train_labels}, columns=col_names)
val = pd.DataFrame({'text':val_texts, 'category':val_labels}, columns=col_names)

In [13]:
data_lm = TextLMDataBunch.from_df('/', train_df=train, valid_df=val)

In [14]:
data_lm.show_batch()

idx,text
0,xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj
1,culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture
2,xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos
3,xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj
4,kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj kesehatan xxbos xxmaj kesehatan xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture xxbos xxmaj culture


In [15]:
# Classifier model data
data_class = TextClasDataBunch.from_df(path = path, train_df =train, valid_df =val, vocab=data_lm.train_ds.vocab, bs=22)

/usr/local/lib/python3.7/dist-packages/fastai/data_block.py:541: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
juli warga miskin gakin malang raya manfaat layan jamin sehat masyarakat jamkesmas jamkesmas ganti asuransi sehat rakyat miskin askeskin sebelumnyamulai juli layan sehat masyarakat butuh jamkesmas jamkesmas masyarakat bebas biaya awat sesuai tentu tetap perintah direktur rumah sakit saiful anwar rssa malang malang raya total jiwa terima jamkesmas bagi kota malang jiwa kota batu jiwa kabupaten malang jiwakuota jamkesmas tentu perintah pusat dasar data bps tetap sk bupati wali kota daerah pawik masyarakat nilai miskin bebas biaya kesehatanmenurut kepala cabang pt askes kota malang dr roni kurnia masyarakat milik kartu jamkesmas data depkes ri manfaat obat gratis jamkesmasada kota malang kuota jamkesmas orang orang biaya perintah pusat untung pemkot malang tanggung b

In [16]:
learn = language_model_learner(data_lm, arch=AWD_LSTM,drop_mult=0.3)
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.216100,3.356262,0.585045,00:00
1,3.173298,3.189869,0.591518,00:00
2,2.896621,3.077504,0.640402,00:00


In [17]:
learn.save_encoder('ft_enc')

In [18]:
classifier = text_classifier_learner(data_class, arch=AWD_LSTM, drop_mult=0.3)
classifier.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (26 items)
x: TextList
xxbos xxmaj culture,xxbos xxmaj kesehatan,xxbos xxmaj culture,xxbos xxmaj culture,xxbos xxmaj culture
y: CategoryList
keluarga raja duga dapat harga mahal milik arti maksud tersendiriseperti lansir anggota keluarga raja kate middleton ratu elizabeth pangeran harry meghan markle istimewa penasaran sajapangeran harry aktris meghan markle tunang rencana khusus raya natal pangeran harry ajak tunang libur calon keluarga baru berita berita baru update trsesaat meghan markle status tunang pangeran harry hadiah apron apron dapat pangeran harrykotak meruapakn hadiah ratu kotak emas gambar burung heron depan sembah jepangpada april obama hadiah ipod ratu eliabeth ipod isi pidato inagurasi obamabukan emas pangeran harry ratu elizabeth unik tulisanaint life a bitch hadiah natalpangeran harry kate middleton hadiah kejut main cleveland cavaliers basket tulis cambridge georgeratu elizabeth hadiah boneka wayang golek kunjung n

In [19]:
classifier.fit_one_cycle(4, 1e-2)